In [58]:
from serde import serde
from serde.json import to_json, from_json
from dataclasses import dataclass
from time import sleep
from scrapping import Voiture

# Récuperation du json et conversion en quelque chose d'utilisable

In [59]:
with open("donnees.json", "r") as fichier:
    contenu_fichier = fichier.read()

In [60]:
contenu_fichier


'[{"modele":"208","carburant":"Essence","prix":17990,"kilometrage":8556,"garantie_kilometrage":"n","boite_de_vitesse":"Manuelle","transmission":"2","couleur":"Blanc","garantie":"SPOTICAR PREMIUM","date_mise_circulation":2022,"puissance":75,"silhouette":"Citadine","nb_places":5,"utilisation_prec":"Ex-Loueur","puissance_fiscale":4,"critair":"1","ptac":1510,"nb_portes":5},{"modele":"RIFTER","carburant":"Diesel","prix":27490,"kilometrage":51739,"garantie_kilometrage":"g","boite_de_vitesse":"Manuelle","transmission":"2","couleur":"Brun","garantie":"SPOTICAR PREMIUM","date_mise_circulation":2020,"puissance":131,"silhouette":"Familiale","nb_places":5,"utilisation_prec":"Ex-Particulier","puissance_fiscale":7,"critair":"2","ptac":2135,"nb_portes":5},{"modele":"208","carburant":"Essence","prix":18970,"kilometrage":40071,"garantie_kilometrage":"n","boite_de_vitesse":"Manuelle","transmission":"2","couleur":"Gris","garantie":"SPOTICAR PREMIUM","date_mise_circulation":2021,"puissance":102,"silhouett

In [61]:
reconstitution = from_json(list[Voiture],contenu_fichier)



In [62]:
reconstitution[0]

Voiture(modele='208', carburant='Essence', prix=17990, kilometrage=8556, garantie_kilometrage='n', boite_de_vitesse='Manuelle', transmission='2', couleur='Blanc', garantie='SPOTICAR PREMIUM', date_mise_circulation=2022, puissance=75, silhouette='Citadine', nb_places=5, utilisation_prec='Ex-Loueur', puissance_fiscale=4, critair='1', ptac=1510, nb_portes=5)

# Conversion en base de données panda et nettoyage

In [63]:
import pandas as pd

In [64]:
df = pd.DataFrame(reconstitution)

In [65]:
df.describe()

,prix,kilometrage,date_mise_circulation,puissance,nb_places,puissance_fiscale,ptac
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,27001.200000,28036.790000,2020.880000,123.630000,5.070000,6.290000,1843.260000
std,7129.965968,24968.844017,1.112645,21.570956,0.670519,1.437555,253.874487
min,14990.000000,5.000000,2018.000000,75.000000,2.000000,4.000000,1510.000000
25%,21620.000000,9407.000000,2020.750000,102.000000,5.000000,5.000000,1641.250000
50%,26370.000000,21772.000000,2021.000000,131.000000,5.000000,6.500000,1770.000000
75%,30235.000000,44125.000000,2022.000000,131.000000,5.000000,7.000000,1950.000000
max,48970.000000,99000.000000,2022.000000,181.000000,9.000000,10.000000,3030.000000


In [66]:
df.head()

,modele,carburant,prix,kilometrage,garantie_kilometrage,boite_de_vitesse,transmission,couleur,garantie,date_mise_circulation,puissance,silhouette,nb_places,utilisation_prec,puissance_fiscale,critair,ptac,nb_portes
0,208,Essence,17990,8556,n,Manuelle,2,Blanc,SPOTICAR PREMIUM,2022,75,Citadine,5,Ex-Loueur,4,1,1510,5
1,RIFTER,Diesel,27490,51739,g,Manuelle,2,Brun,SPOTICAR PREMIUM,2020,131,Familiale,5,Ex-Particulier,7,2,2135,5
2,208,Essence,18970,40071,n,Manuelle,2,Gris,SPOTICAR PREMIUM,2021,102,Citadine,5,Ex-Loueur,5,1,1595,5
3,2008,Essence,26970,15512,n,Automatique,2,Gris,SPOTICAR PREMIUM,2021,131,SUV-4x4,5,Ex-Loueur,7,1,1752,5
4,3008,Diesel,22970,73499,n,Manuelle,2,Rouge,SPOTICAR PREMIUM,2019,130,SUV-4x4,5,Ex-Loueur,6,2,1970,5


In [67]:
df.dtypes

modele                   object
carburant                object
prix                      int64
kilometrage               int64
garantie_kilometrage     object
boite_de_vitesse         object
transmission             object
couleur                  object
garantie                 object
date_mise_circulation     int64
puissance                 int64
silhouette               object
nb_places                 int64
utilisation_prec         object
puissance_fiscale         int64
critair                  object
ptac                      int64
nb_portes                object
dtype: object

In [68]:
for i in df.columns:
    col_selec = f"df.{i}.unique()"
    
    print(f"{i} : {eval(col_selec)} ")
# retourne toutes les modalités pour chaque variable

modele : ['208' 'RIFTER' '2008' '3008' '308' '5008' '508 SW' 'EXPERT' '308 SW'
 'PARTNER' 'TRAVELLER'] 
carburant : ['Essence' 'Diesel' 'Hybride rechargeable' 'Electrique'] 
prix : [17990 27490 18970 26970 22970 21470 23770 20470 17970 22770 24470 22470
 30970 27470 24970 27970 28470 21670 19970 43970 45470 29970 38470 29470
 20970 20770 17470 28970 31470 31970 37970 36770 21970 27770 38770 28770
 16970 15770 15470 33470 25970 19470 23470 26770 31770 33770 32970 39970
 37470 35970 18770 19770 44970 34970 18470 48970 27990 14990 29990 23990] 
kilometrage : [ 8556 51739 40071 15512 73499 25086 24416 30583 49727 19165  8126 16818
 21919 45800 16793 19281 99000 22422 33219 59795    10  8436 26370 31037
 19388 10212 31335 22280  3000 21625 15713 10000 23199 51640 25000 75000
  9461 44500 33000 77125 48997 20743  2762 38840 97034 75249 18267 18340
 51306 89000  9245 14461  4486 22091  9990 69728 48662 31500 20975  7111
  2473 19000     5  5436 24139 45159 21272 44000 32538 10013 15452 22001


## Remarques sur les variables :

- modele : enlever les espaces, ou supprimer les variables peu occurentes (<=1)
- carburant : même remarque
- même remarque pour toutes les variables textuelles
- utilisation précédente : regrouper par pro/loueur/part/ ne sait pas ?

In [37]:
df.convert_dtypes().dtypes #censé convertir automatiquement vers le type le plus adapté
# mais pas convaincu

modele                   string
carburant                string
prix                      Int64
kilometrage               Int64
garantie_kilometrage     string
boite_de_vitesse         string
transmission             string
couleur                  string
garantie                 string
date_mise_circulation     Int64
puissance                 Int64
silhouette               string
nb_places                 Int64
utilisation_prec         string
puissance_fiscale         Int64
critair                  string
ptac                      Int64
nb_portes                object
dtype: object

In [70]:
#df.garantie_kilometrage= df.garantie_kilometrage.astype('bool')
#df.modele= df.garantie_kilometrage.astype('category')
df = df.astype({ "modele": 'object',"carburant": 'category',
                "prix": 'int32',
                "kilometrage" : 'int32',
#                "garantie_kilometrage":'bool',
#                "boite_de_vitesse": 'bool', #checker sur plus grosses données si il existe d'autres modalités
                "transmission" : 'category',
                "couleur" : 'category',
                "garantie" : 'category',
                "date_mise_circulation":'category',
                "puissance" : "int32",
                "silhouette" : 'category',
                "nb_places" : 'int32',
                "utilisation_prec" : 'category',
                "puissance_fiscale" : 'int32',
                "critair": 'category',
                "boite_de_vitesse": 'category',
                "ptac" : "int32"
                })
# convertit tous les types manuellement

In [71]:
df.dtypes # checke les nouveaux types

modele                     object
carburant                category
prix                        int32
kilometrage                 int32
garantie_kilometrage       object
boite_de_vitesse         category
transmission             category
couleur                  category
garantie                 category
date_mise_circulation    category
puissance                   int32
silhouette               category
nb_places                   int32
utilisation_prec         category
puissance_fiscale           int32
critair                  category
ptac                        int32
nb_portes                  object
dtype: object

In [72]:
df.nb_portes.str.match('NA').value_counts() #problème de changement de type avec les na


True    1
Name: nb_portes, dtype: int64

In [16]:
#df.garantie_kilometrage.replace({'n': 'no', 'g': 'yes'})

In [73]:
df['garantie_kilometrage'] = df['garantie_kilometrage'].map({'n':False,'g':True})
# passe garantie km en bool

In [18]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(df)
# sort tout le df

       modele             carburant   prix  kilometrage  garantie_kilometrage  \
0         208               Essence  17990         8556                 False   
1      RIFTER                Diesel  27490        51739                  True   
2         208               Essence  18970        40071                 False   
3        2008               Essence  26970        15512                 False   
4        3008                Diesel  22970        73499                 False   
5         208               Essence  21470        25086                 False   
6         208               Essence  23770        24416                 False   
7         208                Diesel  20470        30583                 False   
8         208               Essence  17970        49727                 False   
9         208               Essence  22770        19165                 False   
10        208               Essence  24470         8126                 False   
11        208               

In [74]:
df.dtypes # types finaux

modele                     object
carburant                category
prix                        int32
kilometrage                 int32
garantie_kilometrage         bool
boite_de_vitesse         category
transmission             category
couleur                  category
garantie                 category
date_mise_circulation    category
puissance                   int32
silhouette               category
nb_places                   int32
utilisation_prec         category
puissance_fiscale           int32
critair                  category
ptac                        int32
nb_portes                  object
dtype: object

## Création de dummys


In [75]:
df.head()

,modele,carburant,prix,kilometrage,garantie_kilometrage,boite_de_vitesse,transmission,couleur,garantie,date_mise_circulation,puissance,silhouette,nb_places,utilisation_prec,puissance_fiscale,critair,ptac,nb_portes
0,208,Essence,17990,8556,False,Manuelle,2,Blanc,SPOTICAR PREMIUM,2022,75,Citadine,5,Ex-Loueur,4,1,1510,5
1,RIFTER,Diesel,27490,51739,True,Manuelle,2,Brun,SPOTICAR PREMIUM,2020,131,Familiale,5,Ex-Particulier,7,2,2135,5
2,208,Essence,18970,40071,False,Manuelle,2,Gris,SPOTICAR PREMIUM,2021,102,Citadine,5,Ex-Loueur,5,1,1595,5
3,2008,Essence,26970,15512,False,Automatique,2,Gris,SPOTICAR PREMIUM,2021,131,SUV-4x4,5,Ex-Loueur,7,1,1752,5
4,3008,Diesel,22970,73499,False,Manuelle,2,Rouge,SPOTICAR PREMIUM,2019,130,SUV-4x4,5,Ex-Loueur,6,2,1970,5


In [76]:
df.dtypes


modele                     object
carburant                category
prix                        int32
kilometrage                 int32
garantie_kilometrage         bool
boite_de_vitesse         category
transmission             category
couleur                  category
garantie                 category
date_mise_circulation    category
puissance                   int32
silhouette               category
nb_places                   int32
utilisation_prec         category
puissance_fiscale           int32
critair                  category
ptac                        int32
nb_portes                  object
dtype: object

In [79]:
df = pd.get_dummies(df)

In [80]:
df.head()

,prix,kilometrage,garantie_kilometrage,puissance,nb_places,puissance_fiscale,ptac,modele_2008,modele_208,modele_3008,...,utilisation_prec_Ex-Société,utilisation_prec_NA,utilisation_prec_Véhicule de direction,critair_0,critair_1,critair_2,nb_portes_3,nb_portes_4,nb_portes_5,nb_portes_NA
0,17990,8556,False,75,5,4,1510,0,1,0,...,0,0,0,0,1,0,0,0,1,0
1,27490,51739,True,131,5,7,2135,0,0,0,...,0,0,0,0,0,1,0,0,1,0
2,18970,40071,False,102,5,5,1595,0,1,0,...,0,0,0,0,1,0,0,0,1,0
3,26970,15512,False,131,5,7,1752,1,0,0,...,0,0,0,0,1,0,0,0,1,0
4,22970,73499,False,130,5,6,1970,0,0,1,...,0,0,0,0,0,1,0,0,1,0


In [81]:
df.columns

Index(['prix', 'kilometrage', 'garantie_kilometrage', 'puissance', 'nb_places',
       'puissance_fiscale', 'ptac', 'modele_2008', 'modele_208', 'modele_3008',
       'modele_308', 'modele_308 SW', 'modele_5008', 'modele_508 SW',
       'modele_EXPERT', 'modele_PARTNER', 'modele_RIFTER', 'modele_TRAVELLER',
       'carburant_Diesel', 'carburant_Electrique', 'carburant_Essence',
       'carburant_Hybride rechargeable', 'boite_de_vitesse_Automatique',
       'boite_de_vitesse_Manuelle', 'transmission_2', 'couleur_Blanc',
       'couleur_Bleu', 'couleur_Brun', 'couleur_Gris', 'couleur_Jaune',
       'couleur_NA', 'couleur_Noir', 'couleur_Rouge', 'couleur_Vert',
       'garantie_SPOTICAR PREMIUM', 'date_mise_circulation_2018',
       'date_mise_circulation_2019', 'date_mise_circulation_2020',
       'date_mise_circulation_2021', 'date_mise_circulation_2022',
       'silhouette_Berline', 'silhouette_Break', 'silhouette_Citadine',
       'silhouette_Familiale', 'silhouette_SUV-4x4', 'silhoue

## Passage sous numpy + ML

In [82]:
y = df['prix'].to_numpy()
X = df.loc[:, df.columns != 'prix'].to_numpy() # tout sauf le prix

In [83]:
type(y), y.shape

(numpy.ndarray, (100,))

In [84]:
type(X), X.shape

(numpy.ndarray, (100, 59))

In [85]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import OneHotEncoder

In [86]:
X_tr, X_te, y_tr, y_te = train_test_split(X, y)

In [88]:
en = ElasticNet()
en_gs = GridSearchCV(
    en,
    {
        "alpha": [2 ** p  for p in range(-6, 6)],
        "l1_ratio": (0.01, 0.25, 0.5, 0.75, 1),
    }
)
en_gs.fit(X_tr, y_tr) #problème vient du fit
en_df = pd.DataFrame(en_gs.cv_results_)
en_df.head()
        

c:\Users\bapti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.069e+07, tolerance: 2.407e+05
  model = cd_fast.enet_coordinate_descent(
c:\Users\bapti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.154e+07, tolerance: 2.427e+05
  model = cd_fast.enet_coordinate_descent(
c:\Users\bapti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the sca

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_l1_ratio,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.013281,0.014528,0.001004,0.000633,0.015625,0.01,"{'alpha': 0.015625, 'l1_ratio': 0.01}",0.899026,0.909088,0.923902,0.852193,0.822549,0.881352,0.037948,16
1,0.003784,0.000755,0.000407,0.000499,0.015625,0.25,"{'alpha': 0.015625, 'l1_ratio': 0.25}",0.895013,0.908253,0.926396,0.852178,0.809445,0.878257,0.042221,17
2,0.002792,0.000394,0.000404,0.000494,0.015625,0.5,"{'alpha': 0.015625, 'l1_ratio': 0.5}",0.887046,0.906110,0.928594,0.854244,0.791604,0.873520,0.047666,21
3,0.005351,0.000652,0.000387,0.000475,0.015625,0.75,"{'alpha': 0.015625, 'l1_ratio': 0.75}",0.868392,0.900944,0.929243,0.863479,0.767180,0.865847,0.054769,25
4,0.009233,0.001742,0.000600,0.000490,0.015625,1,"{'alpha': 0.015625, 'l1_ratio': 1}",0.766879,0.860665,0.898833,0.839333,0.651731,0.803488,0.087195,50


In [89]:
en_gs.best_params_, en_gs.best_score_

({'alpha': 0.25, 'l1_ratio': 0.75}, 0.8900932473748604)

In [90]:
from sklearn.neighbors import KNeighborsRegressor

In [91]:
knr = KNeighborsRegressor()
knr_gs = GridSearchCV(
    knr,
    {
        "n_neighbors": range(5, 15),
        "weights": ("uniform", "distance"),
    }
)

In [92]:
knr_gs.fit(X_tr, y_tr)
knr_df = pd.DataFrame(knr_gs.cv_results_)
knr_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,param_weights,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.000406,0.000497,0.028185,0.048268,5,uniform,"{'n_neighbors': 5, 'weights': 'uniform'}",0.009290,0.520111,0.248784,0.071226,-0.005048,0.168873,0.197486,16
1,0.000801,0.000401,0.004471,0.000552,5,distance,"{'n_neighbors': 5, 'weights': 'distance'}",-0.067306,0.522996,0.354284,-0.236410,0.084468,0.131606,0.275679,20
2,0.000401,0.000491,0.002862,0.000447,6,uniform,"{'n_neighbors': 6, 'weights': 'uniform'}",0.083411,0.564053,0.220184,0.140085,-0.033562,0.194834,0.202258,6
3,0.000200,0.000400,0.004701,0.000601,6,distance,"{'n_neighbors': 6, 'weights': 'distance'}",0.045050,0.576182,0.334779,-0.193007,0.127506,0.178102,0.261151,14
4,0.000325,0.000416,0.002764,0.000484,7,uniform,"{'n_neighbors': 7, 'weights': 'uniform'}",0.052952,0.550424,0.309894,0.179563,-0.213879,0.175791,0.255061,15


In [93]:
knr_gs.best_params_, knr_gs.best_score_

({'n_neighbors': 13, 'weights': 'uniform'}, 0.2296824241983686)

In [94]:
from sklearn.ensemble import RandomForestRegressor

In [95]:
rfr = RandomForestRegressor()
rfr_gs = GridSearchCV(
    rfr,
    {   
        "n_estimators": (8 , 16, 32, 64, 128, 256),
    }
)

In [96]:
rfr_gs.fit(X_tr, y_tr)
rfr_df = pd.DataFrame(rfr_gs.cv_results_)
rfr_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.016837,0.001341,0.001871,0.000928,8,{'n_estimators': 8},0.572895,0.740428,0.740999,0.633865,0.799686,0.697575,0.082162,3
1,0.022348,0.001326,0.001290,0.000405,16,{'n_estimators': 16},0.633290,0.718029,0.535176,0.611315,0.783757,0.656313,0.086348,6
2,0.048178,0.006044,0.002253,0.000735,32,{'n_estimators': 32},0.601908,0.748783,0.639045,0.764479,0.765701,0.703983,0.069444,1
3,0.207315,0.024261,0.009250,0.002212,64,{'n_estimators': 64},0.589947,0.721959,0.673273,0.689953,0.713416,0.677710,0.047127,5
4,0.391023,0.052017,0.020393,0.001189,128,{'n_estimators': 128},0.581334,0.753142,0.649696,0.760653,0.732245,0.695414,0.069354,4


In [97]:
rfr_gs.best_params_, rfr_gs.best_score_

({'n_estimators': 32}, 0.7039832604075377)

In [98]:
from sklearn.svm import SVR

In [100]:
svr = SVR()
svr_gs = GridSearchCV(
    svr,
    {
        "C": (0.1, 1.0, 10),
        "epsilon": (0.1, 1.0, 10),
    }
)
svr_gs.fit(X_tr, y_tr)

svr_df = pd.DataFrame(svr_gs.cv_results_)
svr_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_epsilon,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001174,0.000189,0.001000,0.000542,0.1,0.1,"{'C': 0.1, 'epsilon': 0.1}",-0.066799,-0.013282,-0.191592,-0.168070,-0.158345,-0.119617,0.068007,9
1,0.001386,0.000912,0.000631,0.000516,0.1,1.0,"{'C': 0.1, 'epsilon': 1.0}",-0.066799,-0.013282,-0.191493,-0.167907,-0.158345,-0.119565,0.067963,8
2,0.002142,0.000683,0.001017,0.000030,0.1,10,"{'C': 0.1, 'epsilon': 10}",-0.066799,-0.013282,-0.190505,-0.166225,-0.158345,-0.119031,0.067517,7
3,0.001701,0.000993,0.000599,0.000489,1.0,0.1,"{'C': 1.0, 'epsilon': 0.1}",-0.066213,-0.012567,-0.190645,-0.167037,-0.157306,-0.118754,0.067856,6
4,0.001003,0.000634,0.001102,0.000206,1.0,1.0,"{'C': 1.0, 'epsilon': 1.0}",-0.066213,-0.012567,-0.190617,-0.167037,-0.157306,-0.118748,0.067850,5


In [101]:
svr_gs.best_params_, svr_gs.best_score_

({'C': 10, 'epsilon': 10}, -0.11012313446726747)

In [102]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

In [103]:
pl = Pipeline(
    [
        ("mise_echelle", MinMaxScaler()),
        ("support_vecteurs", SVR()),
    ]
)

In [104]:
pl_gs = GridSearchCV(
    pl,
    {
        "support_vecteurs__C": (0.1, 1.0, 10),
        "support_vecteurs__epsilon": (0.1, 1.0, 10),
    }
)

In [105]:
pl_gs.fit(X_tr, y_tr)

pl_df = pd.DataFrame(pl_gs.cv_results_)
pl_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_support_vecteurs__C,param_support_vecteurs__epsilon,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002205,0.000404,0.002304,0.000402,0.1,0.1,"{'support_vecteurs__C': 0.1, 'support_vecteurs...",-0.066819,-0.013302,-0.191653,-0.168064,-0.158381,-0.119644,0.068014,9
1,0.001708,0.000610,0.002104,0.000206,0.1,1.0,"{'support_vecteurs__C': 0.1, 'support_vecteurs...",-0.066819,-0.013302,-0.191554,-0.167896,-0.158381,-0.119590,0.067969,8
2,0.001800,0.000400,0.002252,0.000386,0.1,10,"{'support_vecteurs__C': 0.1, 'support_vecteurs...",-0.066819,-0.013302,-0.190566,-0.166214,-0.158381,-0.119056,0.067523,6
3,0.002030,0.000519,0.002007,0.000663,1.0,0.1,"{'support_vecteurs__C': 1.0, 'support_vecteurs...",-0.066414,-0.012768,-0.191323,-0.167147,-0.157666,-0.119064,0.067962,7
4,0.001202,0.000400,0.001725,0.000391,1.0,1.0,"{'support_vecteurs__C': 1.0, 'support_vecteurs...",-0.066414,-0.012768,-0.191225,-0.166979,-0.157666,-0.119010,0.067918,5


In [106]:
pl_gs.best_params_, pl_gs.best_score_

({'support_vecteurs__C': 10, 'support_vecteurs__epsilon': 10},
 -0.1126996836316156)

In [110]:
from sklearn.neural_network import MLPRegressor
import numpy as np

In [108]:
pln = Pipeline(
    [
        ("mise_echelle", MinMaxScaler()),
        ("neurones", MLPRegressor()),
    ]
)

In [111]:
pln_gs = GridSearchCV(
    pln,
    {
        "neurones__alpha": 10.0 ** -np.arange(1, 7),
        'neurones__hidden_layer_sizes': ((25,), (50, ), (100,), (20, 20)),
    }
)

In [112]:
pln_gs.fit(X_tr, y_tr)

pln_df = pd.DataFrame(pln_gs.cv_results_)
pln_df.head()

c:\Users\bapti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\bapti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\bapti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\bapti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the opti

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_neurones__alpha,param_neurones__hidden_layer_sizes,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.054616,0.011417,0.000801,0.000747,0.1,"(25,)","{'neurones__alpha': 0.1, 'neurones__hidden_lay...",-12.778884,-13.881388,-12.735643,-37.328183,-28.211921,-20.987204,10.051682,19
1,0.078396,0.015153,0.000798,0.000399,0.1,"(50,)","{'neurones__alpha': 0.1, 'neurones__hidden_lay...",-12.766830,-13.874439,-12.732324,-37.258611,-28.171285,-20.960698,10.026716,13
2,0.161377,0.055340,0.001193,0.000821,0.1,"(100,)","{'neurones__alpha': 0.1, 'neurones__hidden_lay...",-12.727471,-13.829053,-12.708858,-37.186083,-28.120128,-20.914319,10.012483,8
3,0.082910,0.022947,0.000811,0.000406,0.1,"(20, 20)","{'neurones__alpha': 0.1, 'neurones__hidden_lay...",-12.727487,-13.812165,-12.660298,-37.077630,-28.016676,-20.858851,9.972707,5
4,0.142507,0.009790,0.000998,0.000004,0.01,"(25,)","{'neurones__alpha': 0.01, 'neurones__hidden_la...",-12.783087,-13.889717,-12.747487,-37.345546,-28.214652,-20.996098,10.053912,22


In [113]:
pln_gs.best_params_, pln_gs.best_score_

({'neurones__alpha': 1e-05, 'neurones__hidden_layer_sizes': (20, 20)},
 -20.83617692830923)